In [ ]:
import random
import plotly.graph_objects as go
#random.seed(99)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt,six
import scipy
from scipy.stats.stats import pearsonr
import dask.dataframe as dd
import seaborn as sns
import dask.datasets
from dask_sql import Context
import datetime
import time
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None
import findspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, BooleanType
from pyspark.sql.functions import udf, coalesce, col, lit, desc
import pyspark.sql.functions as f
import docx
from docx.shared import Cm


def count_users(name, *list_of_df):
    '''
    Рассчитывает количество уникальных пользователей по каждой рубрике.
    Принимает на вход название сохраняемого файла и таблицы, по которым производится расчет.
    
    '''
    df_all = dd.concat(list(list_of_df))
    users_itogo = df_all.groupby('rubricName').user.nunique().compute()
    users_itogo_result = df_all.user.nunique().compute()

    df20_result_cnt = df20_raw.user.nunique().compute()
    df21_result_cnt = df21_raw.user.nunique().compute()
    df22_result_cnt = df22_raw.user.nunique().compute()
    df23_result_cnt = df23_raw.user.nunique().compute()

    print(users_itogo_result, 
    df20_result_cnt,
    df21_result_cnt,
    df22_result_cnt,
    df23_result_cnt,)

    users_itogo.to_excel('{}users_itogo.xlsx'.format(name))

def agg_df(df, year):
    start_dt = datetime.datetime.now()
    print('Start:', start_dt)
    df1 = df[['user', 'rubricName', 'date']]
    
    try:
        pd_df1_req_cnt = pd.read_excel(f'df1_req_cnt_{year}.xlsx')
        pd_df1_user_cnt = pd.read_excel(f'df1_user_cnt_{year}.xlsx')
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_index=True, right_index=True)
    except:
        df1_user_cnt = df1.groupby('rubricName').user.nunique().compute()
        df1_req_cnt = df1.groupby('rubricName').date.count().compute()
        df1_req_cnt.to_excel(f'df1_req_cnt_{year}.xlsx')
        df1_user_cnt.to_excel(f'df1_user_cnt_{year}.xlsx')
        pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
        pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    
    df1_agg = df1_agg.sort_values('date', ascending=False)
    df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    df1_agg.to_excel(f'df1_agg_{year}.xlsx', index=False)
    end_dt = datetime.datetime.now()
    print('End:', end_dt)
    print('Duration:', end_dt-start_dt)
    return df1_agg

def row_number(df, col, sort_col, index, desc=True, filter_df=True, filter_zero=False):
    if col not in df.columns:
        df[col] = ''
    if filter_df:
        rows_to_count = df[df[col]].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    elif filter_zero:
        rows_to_count = df[df[col]!=0].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    else:
        rows_to_count = df.sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
        df_shape = df.shape
    rows_to_zero = [i for i in df.index if i not in rows_to_count] 
    n = 0
    for r0 in rows_to_zero:
        df.loc[r0, col] = 0
    for r in rows_to_count:
        n += 1
        df.loc[r, col] = n

    return df

def count_ratings(df, index):
# Пользователи
    df_user_max = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].max(axis=1)
    df_user_min = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].min(axis=1)

    df['Рейтинг по пользователям 2020'] = df_user_max == df.loc[:,['Пользователи 2020']].iloc[:,0]
    df['Рейтинг по пользователям 2021'] = df_user_max == df.loc[:,['Пользователи 2021']].iloc[:,0]
    df['Рейтинг по пользователям 2022'] = df_user_max == df.loc[:,['Пользователи 2022']].iloc[:,0]
    df['Рейтинг по пользователям 2023'] = df_user_max == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df = row_number(df, 'Рейтинг по пользователям 2020', 'Пользователи 2020', index=index)
    df = row_number(df, 'Рейтинг по пользователям 2021', 'Пользователи 2021', index=index)
    df = row_number(df, 'Рейтинг по пользователям 2022', 'Пользователи 2022', index=index)
    df = row_number(df, 'Рейтинг по пользователям 2023', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям, итого', 'Пользователи, итого', filter_df=False, index=index)
    
    
    df['Антирейтинг по пользователям 2020'] = df_user_min == df.loc[:,['Пользователи 2020']].iloc[:,0]
    df['Антирейтинг по пользователям 2021'] = df_user_min == df.loc[:,['Пользователи 2021']].iloc[:,0]
    df['Антирейтинг по пользователям 2022'] = df_user_min == df.loc[:,['Пользователи 2022']].iloc[:,0]
    df['Антирейтинг по пользователям 2023'] = df_user_min == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df = row_number(df, 'Антирейтинг по пользователям 2020', 'Пользователи 2020', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по пользователям 2021', 'Пользователи 2021', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по пользователям 2022', 'Пользователи 2022', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по пользователям 2023', 'Пользователи 2023', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по пользователям,\nитого', 'Пользователи, итого', desc=False, filter_df=False, index=index)
    
    
    df['Рейтинг постоянного роста\nпользователей 2020-2023'] = ((df['Пользователи 2020'] < df['Пользователи 2021']) & (df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг постоянного роста\nпользователей 2021-2023'] = ((df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг постоянного роста\nпользователей 2022-2023'] = ((df['Пользователи 2022'] < df['Пользователи 2023']))
    
    df = row_number(df, 'Рейтинг постоянного роста\nпользователей 2020-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nпользователей 2021-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nпользователей 2022-2023', 'Пользователи, итого', index=index)
    
    
    df['Рейтинг постоянного падения\nпользователей 2020-2023'] = ((df['Пользователи 2020'] > df['Пользователи 2021'])  & (df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг постоянного падения\nпользователей 2021-2023'] = ((df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг постоянного падения\nпользователей 2022-2023'] = ((df['Пользователи 2022'] > df['Пользователи 2023']))
    
    df = row_number(df, 'Рейтинг постоянного падения\nпользователей 2020-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nпользователей 2021-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nпользователей 2022-2023', 'Пользователи, итого', index=index)

    df['Рейтинг по пользователям НННВ'] = ((df['Пользователи 2020'] < df['Пользователи 2023'])  & (df['Пользователи 2021'] < df['Пользователи 2023']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг по пользователям ННВВ'] = ((df['Пользователи 2020'] < df['Пользователи 2022'])  & (df['Пользователи 2020'] < df['Пользователи 2023']) & (df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2021'] < df['Пользователи 2023']))
    df['Рейтинг по пользователям НВВВ'] = ((df['Пользователи 2020'] < df['Пользователи 2021'])  & (df['Пользователи 2020'] < df['Пользователи 2022']) & (df['Пользователи 2020'] < df['Пользователи 2023']))
    df['Рейтинг по пользователям ВВВН'] = ((df['Пользователи 2023'] < df['Пользователи 2020'])  & (df['Пользователи 2023'] < df['Пользователи 2022']) & (df['Пользователи 2023'] < df['Пользователи 2021']))
    df['Рейтинг по пользователям ВВНН'] = ((df['Пользователи 2020'] > df['Пользователи 2022'])  & (df['Пользователи 2020'] > df['Пользователи 2023']) & (df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2021'] > df['Пользователи 2023']))
    df['Рейтинг по пользователям ВННН'] = ((df['Пользователи 2020'] > df['Пользователи 2022'])  & (df['Пользователи 2020'] > df['Пользователи 2023']) & (df['Пользователи 2020'] > df['Пользователи 2021']))
    df['Рейтинг по пользователям ВНВН'] = ((df['Пользователи 2020'] > df['Пользователи 2021'])  & (df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг по пользователям НВНВ'] = ((df['Пользователи 2020'] < df['Пользователи 2021'])  & (df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] < df['Пользователи 2023']))

    df = row_number(df, 'Рейтинг по пользователям НННВ', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ННВВ', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям НВВВ', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ВВВН', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ВВНН', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ВННН', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ВНВН', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям НВНВ', 'Пользователи 2023', index=index)
    

    df['Рейтинг стабильности\nчисла пользователей'] = abs(df_user_min / df_user_max - 1)
    df = row_number(df, 'Рейтинг стабильности\nчисла пользователей', 'Пользователи, итого', filter_df=False, index=index)

    
# Запросы    
    df_req_max = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].max(axis=1)
    df_req_min = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].min(axis=1)
    df['Рейтинг по запросам 2020'] = df_req_max == df.loc[:,['Запросы 2020']].iloc[:,0]
    df['Рейтинг по запросам 2021'] = df_req_max == df.loc[:,['Запросы 2021']].iloc[:,0]
    df['Рейтинг по запросам 2022'] = df_req_max == df.loc[:,['Запросы 2022']].iloc[:,0]
    df['Рейтинг по запросам 2023'] = df_req_max == df.loc[:,['Запросы 2023']].iloc[:,0]
    
    df = row_number(df, 'Рейтинг по запросам 2020', 'Запросы 2020', index=index)
    df = row_number(df, 'Рейтинг по запросам 2021', 'Запросы 2021', index=index)
    df = row_number(df, 'Рейтинг по запросам 2022', 'Запросы 2022', index=index)
    df = row_number(df, 'Рейтинг по запросам 2023', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам, итого', 'Запросы, итого', filter_df=False, index=index)

    
    df['Антирейтинг по запросам 2020'] = df_req_min == df.loc[:,['Запросы 2020']].iloc[:,0]
    df['Антирейтинг по запросам 2021'] = df_req_min == df.loc[:,['Запросы 2021']].iloc[:,0]
    df['Антирейтинг по запросам 2022'] = df_req_min == df.loc[:,['Запросы 2022']].iloc[:,0]
    df['Антирейтинг по запросам 2023'] = df_req_min == df.loc[:,['Запросы 2023']].iloc[:,0]
    
    df = row_number(df, 'Антирейтинг по запросам 2020', 'Запросы 2020', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по запросам 2021', 'Запросы 2021', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по запросам 2022', 'Запросы 2022', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по запросам 2023', 'Запросы 2023', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по запросам, итого', 'Запросы, итого', desc=False, filter_df=False, index=index)

    
    df['Рейтинг постоянного роста\nзапросов 2020-2023'] = ((df['Запросы 2020'] < df['Запросы 2021']) & (df['Запросы 2021'] < df['Запросы 2022']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг постоянного роста\nзапросов 2021-2023'] = ((df['Запросы 2021'] < df['Запросы 2022']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг постоянного роста\nзапросов 2022-2023'] = ((df['Запросы 2022'] < df['Запросы 2023']))
    
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов 2020-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов 2021-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов 2022-2023', 'Запросы, итого', index=index)
    
    
    df['Рейтинг постоянного падения\nзапросов 2020-2023'] = ((df['Запросы 2020'] > df['Запросы 2021'])  & (df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг постоянного падения\nзапросов 2021-2023'] = ((df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг постоянного падения\nзапросов 2022-2023'] = ((df['Запросы 2022'] > df['Запросы 2023']))
    
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов 2020-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов 2021-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов 2022-2023', 'Запросы, итого', index=index)
    
    
    df['Рейтинг по запросам НННВ'] = ((df['Запросы 2020'] < df['Запросы 2023'])  & (df['Запросы 2021'] < df['Запросы 2023']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг по запросам ННВВ'] = ((df['Запросы 2020'] < df['Запросы 2022'])  & (df['Запросы 2020'] < df['Запросы 2023']) & (df['Запросы 2021'] < df['Запросы 2022']) & (df['Запросы 2021'] < df['Запросы 2023']))
    df['Рейтинг по запросам НВВВ'] = ((df['Запросы 2020'] < df['Запросы 2021'])  & (df['Запросы 2020'] < df['Запросы 2022']) & (df['Запросы 2020'] < df['Запросы 2023']))
    df['Рейтинг по запросам ВВВН'] = ((df['Запросы 2023'] < df['Запросы 2020'])  & (df['Запросы 2023'] < df['Запросы 2022']) & (df['Запросы 2023'] < df['Запросы 2021']))
    df['Рейтинг по запросам ВВНН'] = ((df['Запросы 2020'] > df['Запросы 2022'])  & (df['Запросы 2020'] > df['Запросы 2023']) & (df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2021'] > df['Запросы 2023']))
    df['Рейтинг по запросам ВННН'] = ((df['Запросы 2020'] > df['Запросы 2022'])  & (df['Запросы 2020'] > df['Запросы 2023']) & (df['Запросы 2020'] > df['Запросы 2021']))
    df['Рейтинг по запросам ВНВН'] = ((df['Запросы 2020'] > df['Запросы 2021']) & (df['Запросы 2021'] < df['Запросы 2022'])& (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг по запросам НВНВ'] = ((df['Запросы 2020'] < df['Запросы 2021'])  & (df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] < df['Запросы 2023']))

    df = row_number(df, 'Рейтинг по запросам НННВ', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ННВВ', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам НВВВ', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ВВВН', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ВВНН', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ВННН', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ВНВН', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам НВНВ', 'Запросы 2023', index=index)
    
    
    df['Рейтинг стабильности\nчисла запросов'] = abs(df_req_min / df_req_max - 1)
    df = row_number(df, 'Рейтинг стабильности\nчисла запросов', 'Запросы, итого', filter_df=False, index=index)
    
    
# Запросов на пользователя
    df_req_user_max = df.loc[:,['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']].max(axis=1)
    df_req_user_min = df.loc[:,['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']].min(axis=1)

    df['Рейтинг запросов на пользователя 2020'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2020']].iloc[:,0]
    df['Рейтинг запросов на пользователя 2021'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2021']].iloc[:,0]
    df['Рейтинг запросов на пользователя 2022'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2022']].iloc[:,0]
    df['Рейтинг запросов на пользователя 2023'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2023']].iloc[:,0]

    df = row_number(df, 'Рейтинг запросов на пользователя 2020', 'Запросов на пользователя 2020', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя 2021', 'Запросов на пользователя 2021', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя 2022', 'Запросов на пользователя 2022', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя 2023', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя, итого', 'Запросов на пользователя, итого', filter_df=False, index=index)


    df['Антирейтинг запросов на пользователя 2020'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2020']].iloc[:,0]
    df['Антирейтинг запросов на пользователя 2021'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2021']].iloc[:,0]
    df['Антирейтинг запросов на пользователя 2022'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2022']].iloc[:,0]
    df['Антирейтинг запросов на пользователя 2023'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2023']].iloc[:,0]

    df = row_number(df, 'Антирейтинг запросов на пользователя 2020', 'Запросов на пользователя 2020', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя 2021', 'Запросов на пользователя 2021', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя 2022', 'Запросов на пользователя 2022', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя 2023', 'Запросов на пользователя 2023', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя, итого', 'Запросов на пользователя, итого', desc=False, filter_df=False, index=index)


    df['Рейтинг постоянного роста\nзапросов на пользователя 2020-2023'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2021']) & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного роста\nзапросов на пользователя 2021-2023'] = ((df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного роста\nзапросов на пользователя 2022-2023'] = ((df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))

    df = row_number(df, 'Рейтинг постоянного роста\nзапросов на пользователя 2020-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов на пользователя 2021-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов на пользователя 2022-2023', 'Запросов на пользователя, итого', index=index)


    df['Рейтинг постоянного падения\nзапросов на пользователя 2020-2023'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного падения\nзапросов на пользователя 2021-2023'] = ((df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного падения\nзапросов на пользователя 2022-2023'] = ((df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))

    df = row_number(df, 'Рейтинг постоянного падения\nзапросов на пользователя 2020-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов на пользователя 2021-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов на пользователя 2022-2023', 'Запросов на пользователя, итого', index=index)

    
    df['Рейтинг запросов на пользователя НННВ'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2023'])  & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2023']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя ННВВ'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2022'])  & (df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2023']) & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя НВВВ'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя ВВВН'] = ((df['Запросов на пользователя 2023'] < df['Запросов на пользователя 2020'])  & (df['Запросов на пользователя 2023'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2023'] < df['Запросов на пользователя 2021']))
    df['Рейтинг запросов на пользователя ВВНН'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2022'])  & (df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2023']) & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя ВННН'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2022'])  & (df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2023']) & (df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2021']))
    df['Рейтинг запросов на пользователя ВНВН'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя НВНВ'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг стабильности\nчисла запросов на пользователя'] = abs(df_user_min / df_user_max - 1)

    df = row_number(df, 'Рейтинг запросов на пользователя НННВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ННВВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя НВВВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ВВВН', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ВВНН', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ВННН', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ВНВН', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя НВНВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг стабильности\nчисла запросов на пользователя', 'Запросов на пользователя, итого', filter_df=False, index=index)
    
    
#Рейтинг схожести динамики
    x = df[['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']]
    y = df[['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Антирейтинг схожести динамики\nпользователей и запросов'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Рейтинг схожести динамики\nпользователей и запросов'] = df['Антирейтинг схожести динамики\nпользователей и запросов']
    df = row_number(df, 'Рейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', filter_df=False, index=index)
    df = row_number(df, 'Антирейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', desc=False, filter_df=False, index=index)
    
    
    x = df[['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']]
    y = df[['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Антирейтинг схожести динамики\nпользователей и запросов на пользователя'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Рейтинг схожести динамики\nпользователей и запросов на пользователя'] = df['Антирейтинг схожести динамики\nпользователей и запросов на пользователя']
    df = row_number(df, 'Рейтинг схожести динамики\nпользователей и запросов на пользователя', 'Запросов на пользователя, итого', filter_df=False, index=index)
    df = row_number(df, 'Антирейтинг схожести динамики\nпользователей и запросов на пользователя', 'Запросов на пользователя, итого', desc=False, filter_df=False, index=index)
    
        
    x = df[['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']]
    y = df[['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Рейтинг схожести динамики\nзапросов и запросов на пользователя'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Антирейтинг схожести динамики\nзапросов и запросов на пользователя'] = df['Рейтинг схожести динамики\nзапросов и запросов на пользователя']
    df = row_number(df, 'Рейтинг схожести динамики\nзапросов и запросов на пользователя', 'Запросов на пользователя, итого', filter_df=False, index=index)
    df = row_number(df, 'Антирейтинг схожести динамики\nзапросов и запросов на пользователя', 'Запросов на пользователя, итого', desc=False, filter_df=False, index=index)
    
    df = df.drop(['№', 'Пользователи 2020', 'Пользователи 2021',
           'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
           'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
           'Запросы 2023', 'Запросы, итого', 'Запросы, %',
           'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
           'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
           'Запросов на пользователя, итого'], axis=1).set_index('Рубрика')
    
    print('Рейтинги построены')
    return df

def horizontal_optimization(table_to_optimize, index):
    new_svod = table_to_optimize
    test_svod = new_svod.copy()
    index_set = set(new_svod.index)
    cnt = 0
    total_sum = 9999
    opt_stats = 0
    iter_stats = 0
    opt_stats2 = 0
    iter_stats2 = 0
    while cnt < 10000 and len(index_set)> 0:
        j = index_set.pop()
        cnt += 1
        cols = [i for i in test_svod.loc[j][test_svod.loc[j] > 0].sort_values(ascending=False).index]

        for i in cols:  
            iter_stats += 1
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, index+1), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, index+1), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
                opt_stats += 1
                
        cols_set = set(cols)
        cnt2 = 0
        while cols_set:
            iter_stats2 += 1
            i = cols_set.pop()
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, index+1), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, index+1), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
                opt_stats2 += 1
        total_sum = test_svod.sum(axis=1).sum()

    test_svod['cnt_horizontal'] = test_svod.drop('cnt_horizontal', axis=1).replace(range(1, index+1), 1).sum(axis=1)
    test_svod = test_svod.sort_values('cnt_horizontal', ascending=False)
    print('Оптимизировано', opt_stats, 'из', iter_stats)
    print('Оптимизировано', opt_stats2, 'из', iter_stats2)
    return test_svod

def vertical_optimization(table_to_optimize, original_table, optimization_level, index):
    df = table_to_optimize.copy()
    new_svod = original_table.copy()
    for i in range(optimization_level):
        opt_stats = 0
        iter_stats = 0
        print('Вертикальная оптимизация шаг {} из {}'.format(i+1, optimization_level))
        for cat_to_optimize in df.index:
            iter_stats += 1
            # сейчас есть значение, превращаем в ноль
            col = df.drop('cnt_horizontal', axis=1).loc[cat_to_optimize].sort_values(ascending=False).index[0]

            # сейчас 0 возвращаем значение из бэкапа
            try:
                cat_to_reset = (
                     df
                     .loc[df.index != cat_to_optimize]
                     .loc[(new_svod.sort_index().loc[:, col] != df.sort_index().loc[:, col]) & (df[col] == 0)]
                     .loc[:,[col, 'cnt_horizontal']]
                     .sort_values([col, 'cnt_horizontal'], ascending=[False, True])
                     .index[0]
                )
                if df.loc[cat_to_reset, 'cnt_horizontal'] + 1 < df.loc[cat_to_optimize, 'cnt_horizontal']:
                    df.loc[cat_to_optimize, col] = 0
                    df.loc[cat_to_reset, col] = new_svod.loc[cat_to_reset, col]
                    df['cnt_horizontal'] = df.drop('cnt_horizontal', axis=1).replace(range(1, index+1), 1).sum(axis=1)
                    opt_stats += 1
            except:
                pass
        print(opt_stats, iter_stats, opt_stats/iter_stats)
    return df

def optimize_df(df_to_optimize, index, cut_level, calc_cut_level=False, optimization_level=10):
    original_svod = df_to_optimize.copy().iloc[:cut_level]
    print(original_svod.shape)
    original_svod_sorted = (
        original_svod
        .fillna(0)
        .set_index('Рубрика')
    ).sort_values('Рубрика')
    original_svod_sorted = original_svod.loc[[not i.isnumeric() for i in original_svod['Рубрика']]]
    
    original_svod_counted = count_ratings(original_svod_sorted, index=index)

    original_svod_counted['cnt_horizontal'] = original_svod_counted.replace(range(1, index+1), 1).sum(axis=1)
    original_svod_counted = original_svod_counted.sort_values('cnt_horizontal', ascending=False)

    hor_optimized_svod = horizontal_optimization(original_svod_counted, index=index)
    vert_optimized_svod = vertical_optimization(hor_optimized_svod, original_svod_counted, optimization_level, index=index)
    
    optimized_svod = vert_optimized_svod.reset_index()
    optimized_svod['В рейтингах, раз'] = optimized_svod['cnt_horizontal']
    optimized_svod = optimized_svod.drop('cnt_horizontal', axis=1).sort_index()

    df = (pd.DataFrame(original_svod_counted.value_counts('cnt_horizontal').rename('Оригинальный файл'))
     .merge(pd.DataFrame(hor_optimized_svod.value_counts('cnt_horizontal').rename('Горизонтальная оптимизация')), left_index=True, right_index=True, how='outer')
     .merge(pd.DataFrame(vert_optimized_svod.value_counts('cnt_horizontal').rename('Вертикальная оптимизация')), left_index=True, right_index=True, how='outer')
     .fillna(0).astype(int))

    display(df)
    
    if calc_cut_level:
        return df
    return optimized_svod

def make_final(original_svod, optimized_svod):
    original_svod_sorted = (
        original_svod
        .fillna(0)
        .set_index('Рубрика')
    ).sort_values('Рубрика')
    original_svod_sorted = original_svod.loc[[not i.isnumeric() for i in original_svod['Рубрика']]]
    
    final_svod = original_svod_sorted.copy().reset_index()

    final_svod = final_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
           'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
           'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
           'Запросы 2023', 'Запросы, итого', 'Запросы, %',
           'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
           'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
           'Запросов на пользователя, итого']].sort_values('№')

    final_svod = final_svod.merge(optimized_svod, left_on='Рубрика', right_on='Рубрика')
    final_svod = final_svod.sort_index().sort_values('№')
    for i in optimized_svod.drop(['Рубрика', 'В рейтингах, раз'], axis=1).columns:
        final_svod = row_number(final_svod, i, i, desc=False, filter_df=False, index=15, filter_zero=True)
    final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
    final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)
    final_svod['Вне рейтингов'] = final_svod['В рейтингах, раз'].fillna(0) == 0
    final_svod = row_number(final_svod, 'Вне рейтингов', 'Запросы, итого', desc=True, filter_df=False, index=15, filter_zero=True)

    return final_svod
    
def rename(df1_req_cnt, df1_user_cnt):
    pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
    pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
    df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    df1_agg = df1_agg.sort_values('date', ascending=False)
    #df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    return df1_agg

#Стартуем с 450 до 750 по 50 пока кол-во 4 не станет = 0
#От найденного значения идем вниз по 10, пока кол-во 4 не станет > 0

def find_cut_level(svod, index, cut_level_num=400, optimization_level=10):
    df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True, optimization_level=optimization_level) 
    while df[df.index > 3]['Вертикальная оптимизация'].sum() > 0:
        cut_level_num += 50
        df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True, optimization_level=optimization_level) 
        
    while df.iloc[3:, 2].sum() == 0 and df.loc[0][2] > 14:
        cut_level_num -= 10
        df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True, optimization_level=optimization_level) 
        
        
    
def show_time(func):
    def wrapped(*args):
        start_dt = datetime.datetime.now()
        res = func(*args)
        end_dt = datetime.datetime.now()
        print('Start:'+' '*4, str(start_dt)[:-7])
        print('End:'+' '*6, str(end_dt)[:-7])
        print('Duration:'+' '*1, str(end_dt-start_dt)[:-7])
        return res
    return wrapped


In [ ]:
@show_time
def count_users_requests(file_path):
    year = file_path.split('_')[-2]
    requests_name = f'Запросы {year}'
    users_name = f'Пользователи {year}'
    df = dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['user', 'rubricName', 'date']]
    cnt_user_grouped = df.groupby('rubricName').user.nunique().to_frame()
    cnt_req_grouped = df.groupby('rubricName').date.count().to_frame()
    user_cnt_year = df.user.nunique().compute()
    print(year, user_cnt_year)
    cnt_user_grouped_year = (cnt_req_grouped
                  .merge(cnt_user_grouped, left_on='rubricName', right_on='rubricName')
                  .rename(columns={'date':requests_name, 'user':users_name})
                  .sort_values(requests_name, ascending=False)
                  .fillna(0)
                  .astype(int)
                  )
    return cnt_user_grouped_year

@show_time
def count_total_users(*files):
    df_list = [dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['rubricName', 'user']] for file_path in files]
    df_all = dd.concat(df_list)
    cnt_user_grouped_total = df_all.groupby('rubricName').user.nunique().to_frame().rename(columns={'user':'Пользователи, итого'})
    cnt_user_total = df_all.user.nunique().compute()
    print(cnt_user_total)
    return cnt_user_grouped_total

@show_time
def make_base_counted_file(*files):
    df = count_users_requests(files[0])
    df.to_csv(f'{files[0]}_count_users_requests.csv', single_file=True)
    for file in files[1:]:
        df = df.merge(count_users_requests(file), left_on='rubricName', right_on='rubricName')
        df.to_csv(f'{file}_count_users_requests.csv', single_file=True)
        
    df = df.merge(count_total_users(files), left_on='rubricName', right_on='rubricName')
    df.to_csv('{}_count_total_users.csv'.format(file.split("\\")[-2]), single_file=True)
    return df


@show_time
def count_users_requests(file_path):
    try:
        cnt_user_grouped_year = dd.read_csv(f'{file_path}_count_users_requests.csv')
        print('read ' + f'{file_path}_count_users_requests.csv')
    except:
        year = file_path.split('_')[-2]
        requests_name = f'Запросы {year}'
        users_name = f'Пользователи {year}'
        df = dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['user', 'rubricName', 'date']]
        cnt_user_grouped = df.groupby('rubricName').user.nunique().to_frame()
        cnt_req_grouped = df.groupby('rubricName').date.count().to_frame()
        #user_cnt_year = df.user.nunique().compute()
        #print(year, user_cnt_year)
        cnt_user_grouped_year = (cnt_req_grouped
                      .merge(cnt_user_grouped, left_on='rubricName', right_on='rubricName')
                      .rename(columns={'date':requests_name, 'user':users_name})
                      .sort_values(requests_name, ascending=False)
                      .fillna(0)
                      .astype(int)
                      )
        cnt_user_grouped_year.to_csv(f'{file_path}_count_users_requests.csv', single_file=True)
    finally:
        df = dd.read_csv(f'{file_path}_count_users_requests.csv')
    return df

@show_time
def count_total_users(*files):
    file_path = "\\".join(files[0][0].split("\\")[:-1]) + "\\" + files[0][0].split("\\")[-2]
    try:
        cnt_user_grouped_total = dd.read_csv("{}_count_total_users.csv".format(file_path))
        print('read ' + "{}_count_total_users.csv".format(file_path))
    except:
        df_list = [dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['rubricName', 'user']] for file_path in files]
        df_all = dd.concat(df_list)
        cnt_user_grouped_total = df_all.groupby('rubricName').user.nunique().to_frame().rename(columns={'user':'Пользователи, итого'})
        #cnt_user_total = df_all.user.nunique().compute()
        #print(cnt_user_total)
        cnt_user_grouped_total.to_csv(f'{file_path}_count_total_users.csv', single_file=True)
    finally:
        df = dd.read_csv("{}_count_total_users.csv".format(file_path))
    return df

@show_time
def make_base_counted_file(*files):
    df = count_users_requests(files[0])
    for file in files[1:]:
        df = df.merge(count_users_requests(file), left_on='rubricName', right_on='rubricName', how='outer')
    df = df.merge(count_total_users(files), left_on='rubricName', right_on='rubricName', how='outer')
    df.to_csv('Almati_master_file.csv', single_file=True)
    return df

from dask.diagnostics import ProgressBar
ProgressBar().register()

In [ ]:
def repack_file_old(file):
    start_t = time.time()
    file_name = str(file).replace('\\', '\\\\')
    print(file_name)
    df = dd.read_csv(file, dtype={'fl': 'object'})
    df = df[['rubricName','time','date','tx', 'user']]
    df['time'] = df['time'].apply(lambda x: x.split(' ')[1].split('.')[0], meta=('time', 'object'))
    df['user'] = df['user'].apply(hash, meta=('user', 'int64'))
    new_file_name = file_name.replace('.csv', '_repacked.csv')
    df.to_csv(new_file_name, single_file=True, index=False)
    end_t = time.time()
    print(end_t-start_t)

def repack_file(file_name):
    new_name = file_name.replace('\\', '\\\\').replace('.csv', '_repacked.csv')
    print(new_name)
    start_t = time.time()
    df_schema = StructType([StructField('user', StringType(), True), StructField('proj', StringType(), True), StructField('rubricName', StringType(), True), StructField('rubricID', StringType(), True), StructField('time', StringType(), True), StructField('date', StringType(), True), StructField('org', StringType(), True), StructField('branch', StringType(), True), StructField('prod', StringType(), True), StructField('tx', StringType(), True), StructField('fl', StringType(), True), StructField('devmod', StringType(), True), StructField('lat', StringType(), True), StructField('lon', StringType(), True)])
    df1 = spark.read.csv(file_name, header='true', schema=df_schema).selectExpr('rubricName','substr(time, 12, 8) as time','date','tx', 'hash(user) user').where('rubricName is not null')
    df1.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").mode('overwrite').csv(new_name)  
    end_t = time.time()
    print(end_t-start_t)

In [13]:
print('Ready:', str(datetime.datetime.now())[:-7])

Ready: 2023-03-25 02:00:56
